# Importing necessary libraries

In [38]:
# importing necessary libraries into the code
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch
import seaborn as sns
import scipy.stats as stats
import re
import warnings
warnings.filterwarnings("ignore")

# Data Preparation
## Import the Dataset

In [39]:
#loading the dataset into the dataframes
old_results = pd.read_csv("race_results.csv")
races = pd.read_csv("races.csv")
circuits = pd.read_csv("circuits.csv")
drivers = pd.read_csv("drivers.csv")
constructors = pd.read_csv("constructors.csv")
pd.set_option('display.max_columns', 75)
pd.set_option('display.max_rows', 100)

### Check if the data was successfully loaded or not

In [40]:
old_results.head()

,season,round,driverId,driverName,constructorId,constructorName,number,position,positionText,points,grid,laps,status,time,fastestLapRank,fastestLap_lap,fastestLapTime,averageSpeed
0,1950,1,farina,Nino Farina,alfa,Alfa Romeo,2.0,1,1,9.0,1,70,Finished,2:13:23.6,NaN,NaN,NaN,NaN
1,1950,1,fagioli,Luigi Fagioli,alfa,Alfa Romeo,3.0,2,2,6.0,2,70,Finished,+2.6,NaN,NaN,NaN,NaN
2,1950,1,reg_parnell,Reg Parnell,alfa,Alfa Romeo,4.0,3,3,4.0,4,70,Finished,+52.0,NaN,NaN,NaN,NaN
3,1950,1,cabantous,Yves Cabantous,lago,Talbot-Lago,14.0,4,4,3.0,6,68,+2 Laps,NaN,NaN,NaN,NaN,NaN
4,1950,1,rosier,Louis Rosier,lago,Talbot-Lago,15.0,5,5,2.0,9,68,+2 Laps,NaN,NaN,NaN,NaN,NaN


In [41]:
races.head()

,season,round,raceName,circuitId,circuitName,date,time,firstPractice,secondPractice,thirdPractice,qualifying,sprint,url
0,1950,1,British Grand Prix,silverstone,Silverstone Circuit,1950-05-13,NaN,NaN,NaN,NaN,NaN,NaN,http://en.wikipedia.org/wiki/1950_British_Gran...
1,1950,2,Monaco Grand Prix,monaco,Circuit de Monaco,1950-05-21,NaN,NaN,NaN,NaN,NaN,NaN,http://en.wikipedia.org/wiki/1950_Monaco_Grand...
2,1950,3,Indianapolis 500,indianapolis,Indianapolis Motor Speedway,1950-05-30,NaN,NaN,NaN,NaN,NaN,NaN,http://en.wikipedia.org/wiki/1950_Indianapolis...
3,1950,4,Swiss Grand Prix,bremgarten,Circuit Bremgarten,1950-06-04,NaN,NaN,NaN,NaN,NaN,NaN,http://en.wikipedia.org/wiki/1950_Swiss_Grand_...
4,1950,5,Belgian Grand Prix,spa,Circuit de Spa-Francorchamps,1950-06-18,NaN,NaN,NaN,NaN,NaN,NaN,http://en.wikipedia.org/wiki/1950_Belgian_Gran...


In [42]:
circuits.head()

,circuitId,circuitName,lat,long,locality,country,url
0,adelaide,Adelaide Street Circuit,-34.9272,138.61700,Adelaide,Australia,http://en.wikipedia.org/wiki/Adelaide_Street_C...
1,ain-diab,Ain Diab,33.5786,-7.68750,Casablanca,Morocco,http://en.wikipedia.org/wiki/Ain-Diab_Circuit
2,aintree,Aintree,53.4769,-2.94056,Liverpool,UK,http://en.wikipedia.org/wiki/Aintree_Motor_Rac...
3,albert_park,Albert Park Grand Prix Circuit,-37.8497,144.96800,Melbourne,Australia,http://en.wikipedia.org/wiki/Melbourne_Grand_P...
4,americas,Circuit of the Americas,30.1328,-97.64110,Austin,USA,http://en.wikipedia.org/wiki/Circuit_of_the_Am...


In [43]:
drivers.head()

,driverId,givenName,familyName,code,permanentNumber,dateOfBirth,nationality,url
0,abate,Carlo,Abate,NaN,NaN,1932-07-10,Italian,http://en.wikipedia.org/wiki/Carlo_Mario_Abate
1,abecassis,George,Abecassis,NaN,NaN,1913-03-21,British,http://en.wikipedia.org/wiki/George_Abecassis
2,acheson,Kenny,Acheson,NaN,NaN,1957-11-27,British,http://en.wikipedia.org/wiki/Kenny_Acheson
3,adams,Philippe,Adams,NaN,NaN,1969-11-19,Belgian,http://en.wikipedia.org/wiki/Philippe_Adams
4,ader,Walt,Ader,NaN,NaN,1913-12-15,American,http://en.wikipedia.org/wiki/Walt_Ader


In [44]:
constructors.head()

,constructorId,constructorName,nationality,url
0,adams,Adams,American,http://en.wikipedia.org/wiki/Adams_(constructor)
1,afm,AFM,German,http://en.wikipedia.org/wiki/Alex_von_Falkenha...
2,ags,AGS,French,http://en.wikipedia.org/wiki/Automobiles_Gonfa...
3,alfa,Alfa Romeo,Swiss,http://en.wikipedia.org/wiki/Alfa_Romeo_in_For...
4,alphatauri,AlphaTauri,Italian,http://en.wikipedia.org/wiki/Scuderia_AlphaTauri


### Merging multiple dataset into one

In [45]:
# merging race results with race dataset to new dataframe (results1)
results1 = old_results.merge(races, on=["season", "round"], how="left")

In [46]:
# checking if the data was successfully merged or not
results1.head()

,season,round,driverId,driverName,constructorId,constructorName,number,position,positionText,points,grid,laps,status,time_x,fastestLapRank,fastestLap_lap,fastestLapTime,averageSpeed,raceName,circuitId,circuitName,date,time_y,firstPractice,secondPractice,thirdPractice,qualifying,sprint,url
0,1950,1,farina,Nino Farina,alfa,Alfa Romeo,2.0,1,1,9.0,1,70,Finished,2:13:23.6,NaN,NaN,NaN,NaN,British Grand Prix,silverstone,Silverstone Circuit,1950-05-13,NaN,NaN,NaN,NaN,NaN,NaN,http://en.wikipedia.org/wiki/1950_British_Gran...
1,1950,1,fagioli,Luigi Fagioli,alfa,Alfa Romeo,3.0,2,2,6.0,2,70,Finished,+2.6,NaN,NaN,NaN,NaN,British Grand Prix,silverstone,Silverstone Circuit,1950-05-13,NaN,NaN,NaN,NaN,NaN,NaN,http://en.wikipedia.org/wiki/1950_British_Gran...
2,1950,1,reg_parnell,Reg Parnell,alfa,Alfa Romeo,4.0,3,3,4.0,4,70,Finished,+52.0,NaN,NaN,NaN,NaN,British Grand Prix,silverstone,Silverstone Circuit,1950-05-13,NaN,NaN,NaN,NaN,NaN,NaN,http://en.wikipedia.org/wiki/1950_British_Gran...
3,1950,1,cabantous,Yves Cabantous,lago,Talbot-Lago,14.0,4,4,3.0,6,68,+2 Laps,NaN,NaN,NaN,NaN,NaN,British Grand Prix,silverstone,Silverstone Circuit,1950-05-13,NaN,NaN,NaN,NaN,NaN,NaN,http://en.wikipedia.org/wiki/1950_British_Gran...
4,1950,1,rosier,Louis Rosier,lago,Talbot-Lago,15.0,5,5,2.0,9,68,+2 Laps,NaN,NaN,NaN,NaN,NaN,British Grand Prix,silverstone,Silverstone Circuit,1950-05-13,NaN,NaN,NaN,NaN,NaN,NaN,http://en.wikipedia.org/wiki/1950_British_Gran...


In [47]:
# merging results1 with circuit dataset to new dataframe (results2)
results2 = results1.merge(circuits, on="circuitId", how="left")

In [48]:
# checking if the data was successfully merged or not
results2.head()

,season,round,driverId,driverName,constructorId,constructorName,number,position,positionText,points,grid,laps,status,time_x,fastestLapRank,fastestLap_lap,fastestLapTime,averageSpeed,raceName,circuitId,circuitName_x,date,time_y,firstPractice,secondPractice,thirdPractice,qualifying,sprint,url_x,circuitName_y,lat,long,locality,country,url_y
0,1950,1,farina,Nino Farina,alfa,Alfa Romeo,2.0,1,1,9.0,1,70,Finished,2:13:23.6,NaN,NaN,NaN,NaN,British Grand Prix,silverstone,Silverstone Circuit,1950-05-13,NaN,NaN,NaN,NaN,NaN,NaN,http://en.wikipedia.org/wiki/1950_British_Gran...,Silverstone Circuit,52.0786,-1.01694,Silverstone,UK,http://en.wikipedia.org/wiki/Silverstone_Circuit
1,1950,1,fagioli,Luigi Fagioli,alfa,Alfa Romeo,3.0,2,2,6.0,2,70,Finished,+2.6,NaN,NaN,NaN,NaN,British Grand Prix,silverstone,Silverstone Circuit,1950-05-13,NaN,NaN,NaN,NaN,NaN,NaN,http://en.wikipedia.org/wiki/1950_British_Gran...,Silverstone Circuit,52.0786,-1.01694,Silverstone,UK,http://en.wikipedia.org/wiki/Silverstone_Circuit
2,1950,1,reg_parnell,Reg Parnell,alfa,Alfa Romeo,4.0,3,3,4.0,4,70,Finished,+52.0,NaN,NaN,NaN,NaN,British Grand Prix,silverstone,Silverstone Circuit,1950-05-13,NaN,NaN,NaN,NaN,NaN,NaN,http://en.wikipedia.org/wiki/1950_British_Gran...,Silverstone Circuit,52.0786,-1.01694,Silverstone,UK,http://en.wikipedia.org/wiki/Silverstone_Circuit
3,1950,1,cabantous,Yves Cabantous,lago,Talbot-Lago,14.0,4,4,3.0,6,68,+2 Laps,NaN,NaN,NaN,NaN,NaN,British Grand Prix,silverstone,Silverstone Circuit,1950-05-13,NaN,NaN,NaN,NaN,NaN,NaN,http://en.wikipedia.org/wiki/1950_British_Gran...,Silverstone Circuit,52.0786,-1.01694,Silverstone,UK,http://en.wikipedia.org/wiki/Silverstone_Circuit
4,1950,1,rosier,Louis Rosier,lago,Talbot-Lago,15.0,5,5,2.0,9,68,+2 Laps,NaN,NaN,NaN,NaN,NaN,British Grand Prix,silverstone,Silverstone Circuit,1950-05-13,NaN,NaN,NaN,NaN,NaN,NaN,http://en.wikipedia.org/wiki/1950_British_Gran...,Silverstone Circuit,52.0786,-1.01694,Silverstone,UK,http://en.wikipedia.org/wiki/Silverstone_Circuit


In [49]:
# merging results2 with drivers dataset to new dataframe (results3)
results3 = results2.merge(drivers, on="driverId", how="left")

In [50]:
# checking if the data was successfully merged or not
results3.head()

,season,round,driverId,driverName,constructorId,constructorName,number,position,positionText,points,grid,laps,status,time_x,fastestLapRank,fastestLap_lap,fastestLapTime,averageSpeed,raceName,circuitId,circuitName_x,date,time_y,firstPractice,secondPractice,thirdPractice,qualifying,sprint,url_x,circuitName_y,lat,long,locality,country,url_y,givenName,familyName,code,permanentNumber,dateOfBirth,nationality,url
0,1950,1,farina,Nino Farina,alfa,Alfa Romeo,2.0,1,1,9.0,1,70,Finished,2:13:23.6,NaN,NaN,NaN,NaN,British Grand Prix,silverstone,Silverstone Circuit,1950-05-13,NaN,NaN,NaN,NaN,NaN,NaN,http://en.wikipedia.org/wiki/1950_British_Gran...,Silverstone Circuit,52.0786,-1.01694,Silverstone,UK,http://en.wikipedia.org/wiki/Silverstone_Circuit,Nino,Farina,NaN,NaN,1906-10-30,Italian,http://en.wikipedia.org/wiki/Nino_Farina
1,1950,1,fagioli,Luigi Fagioli,alfa,Alfa Romeo,3.0,2,2,6.0,2,70,Finished,+2.6,NaN,NaN,NaN,NaN,British Grand Prix,silverstone,Silverstone Circuit,1950-05-13,NaN,NaN,NaN,NaN,NaN,NaN,http://en.wikipedia.org/wiki/1950_British_Gran...,Silverstone Circuit,52.0786,-1.01694,Silverstone,UK,http://en.wikipedia.org/wiki/Silverstone_Circuit,Luigi,Fagioli,NaN,NaN,1898-06-09,Italian,http://en.wikipedia.org/wiki/Luigi_Fagioli
2,1950,1,reg_parnell,Reg Parnell,alfa,Alfa Romeo,4.0,3,3,4.0,4,70,Finished,+52.0,NaN,NaN,NaN,NaN,British Grand Prix,silverstone,Silverstone Circuit,1950-05-13,NaN,NaN,NaN,NaN,NaN,NaN,http://en.wikipedia.org/wiki/1950_British_Gran...,Silverstone Circuit,52.0786,-1.01694,Silverstone,UK,http://en.wikipedia.org/wiki/Silverstone_Circuit,Reg,Parnell,NaN,NaN,1911-07-02,British,http://en.wikipedia.org/wiki/Reg_Parnell
3,1950,1,cabantous,Yves Cabantous,lago,Talbot-Lago,14.0,4,4,3.0,6,68,+2 Laps,NaN,NaN,NaN,NaN,NaN,British Grand Prix,silverstone,Silverstone Circuit,1950-05-13,NaN,NaN,NaN,NaN,NaN,NaN,http://en.wikipedia.org/wiki/1950_British_Gran...,Silverstone Circuit,52.0786,-1.01694,Silverstone,UK,http://en.wikipedia.org/wiki/Silverstone_Circuit,Yves,Cabantous,NaN,NaN,1904-10-08,French,http://en.wikipedia.org/wiki/Yves_Giraud_Caban...
4,1950,1,rosier,Louis Rosier,lago,Talbot-Lago,15.0,5,5,2.0,9,68,+2 Laps,NaN,NaN,NaN,NaN,NaN,British Grand Prix,silverstone,Silverstone Circuit,1950-05-13,NaN,NaN,NaN,NaN,NaN,NaN,http://en.wikipedia.org/wiki/1950_British_Gran...,Silverstone Circuit,52.0786,-1.01694,Silverstone,UK,http://en.wikipedia.org/wiki/Silverstone_Circuit,Louis,Rosier,NaN,NaN,1905-11-05,French,http://en.wikipedia.org/wiki/Louis_Rosier


In [51]:
# renaming column url_x to race_url
results3 = results3.rename(columns={"url_x": "race_url"})

In [52]:
# checking if the renaming was successful
results3.head()

,season,round,driverId,driverName,constructorId,constructorName,number,position,positionText,points,grid,laps,status,time_x,fastestLapRank,fastestLap_lap,fastestLapTime,averageSpeed,raceName,circuitId,circuitName_x,date,time_y,firstPractice,secondPractice,thirdPractice,qualifying,sprint,race_url,circuitName_y,lat,long,locality,country,url_y,givenName,familyName,code,permanentNumber,dateOfBirth,nationality,url
0,1950,1,farina,Nino Farina,alfa,Alfa Romeo,2.0,1,1,9.0,1,70,Finished,2:13:23.6,NaN,NaN,NaN,NaN,British Grand Prix,silverstone,Silverstone Circuit,1950-05-13,NaN,NaN,NaN,NaN,NaN,NaN,http://en.wikipedia.org/wiki/1950_British_Gran...,Silverstone Circuit,52.0786,-1.01694,Silverstone,UK,http://en.wikipedia.org/wiki/Silverstone_Circuit,Nino,Farina,NaN,NaN,1906-10-30,Italian,http://en.wikipedia.org/wiki/Nino_Farina
1,1950,1,fagioli,Luigi Fagioli,alfa,Alfa Romeo,3.0,2,2,6.0,2,70,Finished,+2.6,NaN,NaN,NaN,NaN,British Grand Prix,silverstone,Silverstone Circuit,1950-05-13,NaN,NaN,NaN,NaN,NaN,NaN,http://en.wikipedia.org/wiki/1950_British_Gran...,Silverstone Circuit,52.0786,-1.01694,Silverstone,UK,http://en.wikipedia.org/wiki/Silverstone_Circuit,Luigi,Fagioli,NaN,NaN,1898-06-09,Italian,http://en.wikipedia.org/wiki/Luigi_Fagioli
2,1950,1,reg_parnell,Reg Parnell,alfa,Alfa Romeo,4.0,3,3,4.0,4,70,Finished,+52.0,NaN,NaN,NaN,NaN,British Grand Prix,silverstone,Silverstone Circuit,1950-05-13,NaN,NaN,NaN,NaN,NaN,NaN,http://en.wikipedia.org/wiki/1950_British_Gran...,Silverstone Circuit,52.0786,-1.01694,Silverstone,UK,http://en.wikipedia.org/wiki/Silverstone_Circuit,Reg,Parnell,NaN,NaN,1911-07-02,British,http://en.wikipedia.org/wiki/Reg_Parnell
3,1950,1,cabantous,Yves Cabantous,lago,Talbot-Lago,14.0,4,4,3.0,6,68,+2 Laps,NaN,NaN,NaN,NaN,NaN,British Grand Prix,silverstone,Silverstone Circuit,1950-05-13,NaN,NaN,NaN,NaN,NaN,NaN,http://en.wikipedia.org/wiki/1950_British_Gran...,Silverstone Circuit,52.0786,-1.01694,Silverstone,UK,http://en.wikipedia.org/wiki/Silverstone_Circuit,Yves,Cabantous,NaN,NaN,1904-10-08,French,http://en.wikipedia.org/wiki/Yves_Giraud_Caban...
4,1950,1,rosier,Louis Rosier,lago,Talbot-Lago,15.0,5,5,2.0,9,68,+2 Laps,NaN,NaN,NaN,NaN,NaN,British Grand Prix,silverstone,Silverstone Circuit,1950-05-13,NaN,NaN,NaN,NaN,NaN,NaN,http://en.wikipedia.org/wiki/1950_British_Gran...,Silverstone Circuit,52.0786,-1.01694,Silverstone,UK,http://en.wikipedia.org/wiki/Silverstone_Circuit,Louis,Rosier,NaN,NaN,1905-11-05,French,http://en.wikipedia.org/wiki/Louis_Rosier


In [53]:
# merging results3 with constructors dataset to new dataframe (results)
results = results3.merge(constructors, on="constructorId", how="left")

In [54]:
# checking if the data was successfully merged or not
results.head()

,season,round,driverId,driverName,constructorId,constructorName_x,number,position,positionText,points,grid,laps,status,time_x,fastestLapRank,fastestLap_lap,fastestLapTime,averageSpeed,raceName,circuitId,circuitName_x,date,time_y,firstPractice,secondPractice,thirdPractice,qualifying,sprint,race_url,circuitName_y,lat,long,locality,country,url_y,givenName,familyName,code,permanentNumber,dateOfBirth,nationality_x,url_x,constructorName_y,nationality_y,url_y
0,1950,1,farina,Nino Farina,alfa,Alfa Romeo,2.0,1,1,9.0,1,70,Finished,2:13:23.6,NaN,NaN,NaN,NaN,British Grand Prix,silverstone,Silverstone Circuit,1950-05-13,NaN,NaN,NaN,NaN,NaN,NaN,http://en.wikipedia.org/wiki/1950_British_Gran...,Silverstone Circuit,52.0786,-1.01694,Silverstone,UK,http://en.wikipedia.org/wiki/Silverstone_Circuit,Nino,Farina,NaN,NaN,1906-10-30,Italian,http://en.wikipedia.org/wiki/Nino_Farina,Alfa Romeo,Swiss,http://en.wikipedia.org/wiki/Alfa_Romeo_in_For...
1,1950,1,fagioli,Luigi Fagioli,alfa,Alfa Romeo,3.0,2,2,6.0,2,70,Finished,+2.6,NaN,NaN,NaN,NaN,British Grand Prix,silverstone,Silverstone Circuit,1950-05-13,NaN,NaN,NaN,NaN,NaN,NaN,http://en.wikipedia.org/wiki/1950_British_Gran...,Silverstone Circuit,52.0786,-1.01694,Silverstone,UK,http://en.wikipedia.org/wiki/Silverstone_Circuit,Luigi,Fagioli,NaN,NaN,1898-06-09,Italian,http://en.wikipedia.org/wiki/Luigi_Fagioli,Alfa Romeo,Swiss,http://en.wikipedia.org/wiki/Alfa_Romeo_in_For...
2,1950,1,reg_parnell,Reg Parnell,alfa,Alfa Romeo,4.0,3,3,4.0,4,70,Finished,+52.0,NaN,NaN,NaN,NaN,British Grand Prix,silverstone,Silverstone Circuit,1950-05-13,NaN,NaN,NaN,NaN,NaN,NaN,http://en.wikipedia.org/wiki/1950_British_Gran...,Silverstone Circuit,52.0786,-1.01694,Silverstone,UK,http://en.wikipedia.org/wiki/Silverstone_Circuit,Reg,Parnell,NaN,NaN,1911-07-02,British,http://en.wikipedia.org/wiki/Reg_Parnell,Alfa Romeo,Swiss,http://en.wikipedia.org/wiki/Alfa_Romeo_in_For...
3,1950,1,cabantous,Yves Cabantous,lago,Talbot-Lago,14.0,4,4,3.0,6,68,+2 Laps,NaN,NaN,NaN,NaN,NaN,British Grand Prix,silverstone,Silverstone Circuit,1950-05-13,NaN,NaN,NaN,NaN,NaN,NaN,http://en.wikipedia.org/wiki/1950_British_Gran...,Silverstone Circuit,52.0786,-1.01694,Silverstone,UK,http://en.wikipedia.org/wiki/Silverstone_Circuit,Yves,Cabantous,NaN,NaN,1904-10-08,French,http://en.wikipedia.org/wiki/Yves_Giraud_Caban...,Talbot-Lago,French,http://en.wikipedia.org/wiki/Talbot-Lago
4,1950,1,rosier,Louis Rosier,lago,Talbot-Lago,15.0,5,5,2.0,9,68,+2 Laps,NaN,NaN,NaN,NaN,NaN,British Grand Prix,silverstone,Silverstone Circuit,1950-05-13,NaN,NaN,NaN,NaN,NaN,NaN,http://en.wikipedia.org/wiki/1950_British_Gran...,Silverstone Circuit,52.0786,-1.01694,Silverstone,UK,http://en.wikipedia.org/wiki/Silverstone_Circuit,Louis,Rosier,NaN,NaN,1905-11-05,French,http://en.wikipedia.org/wiki/Louis_Rosier,Talbot-Lago,French,http://en.wikipedia.org/wiki/Talbot-Lago


In [55]:
# checking the shape of the created dataframe results
results.shape

(26759, 45)

In [56]:
# checking the columns of the dataframe results 
results.columns

Index(['season', 'round', 'driverId', 'driverName', 'constructorId',
       'constructorName_x', 'number', 'position', 'positionText', 'points',
       'grid', 'laps', 'status', 'time_x', 'fastestLapRank', 'fastestLap_lap',
       'fastestLapTime', 'averageSpeed', 'raceName', 'circuitId',
       'circuitName_x', 'date', 'time_y', 'firstPractice', 'secondPractice',
       'thirdPractice', 'qualifying', 'sprint', 'race_url', 'circuitName_y',
       'lat', 'long', 'locality', 'country', 'url_y', 'givenName',
       'familyName', 'code', 'permanentNumber', 'dateOfBirth', 'nationality_x',
       'url_x', 'constructorName_y', 'nationality_y', 'url_y'],
      dtype='object')

# Create a new column "driverExperience" as the years of experience the driver has.

In [57]:
results['driverExperience'] = (
    results['season'] - results.groupby('driverId')['season'].transform('min')
)

In [58]:
results.head()

,season,round,driverId,driverName,constructorId,constructorName_x,number,position,positionText,points,grid,laps,status,time_x,fastestLapRank,fastestLap_lap,fastestLapTime,averageSpeed,raceName,circuitId,circuitName_x,date,time_y,firstPractice,secondPractice,thirdPractice,qualifying,sprint,race_url,circuitName_y,lat,long,locality,country,url_y,givenName,familyName,code,permanentNumber,dateOfBirth,nationality_x,url_x,constructorName_y,nationality_y,url_y,driverExperience
0,1950,1,farina,Nino Farina,alfa,Alfa Romeo,2.0,1,1,9.0,1,70,Finished,2:13:23.6,NaN,NaN,NaN,NaN,British Grand Prix,silverstone,Silverstone Circuit,1950-05-13,NaN,NaN,NaN,NaN,NaN,NaN,http://en.wikipedia.org/wiki/1950_British_Gran...,Silverstone Circuit,52.0786,-1.01694,Silverstone,UK,http://en.wikipedia.org/wiki/Silverstone_Circuit,Nino,Farina,NaN,NaN,1906-10-30,Italian,http://en.wikipedia.org/wiki/Nino_Farina,Alfa Romeo,Swiss,http://en.wikipedia.org/wiki/Alfa_Romeo_in_For...,0
1,1950,1,fagioli,Luigi Fagioli,alfa,Alfa Romeo,3.0,2,2,6.0,2,70,Finished,+2.6,NaN,NaN,NaN,NaN,British Grand Prix,silverstone,Silverstone Circuit,1950-05-13,NaN,NaN,NaN,NaN,NaN,NaN,http://en.wikipedia.org/wiki/1950_British_Gran...,Silverstone Circuit,52.0786,-1.01694,Silverstone,UK,http://en.wikipedia.org/wiki/Silverstone_Circuit,Luigi,Fagioli,NaN,NaN,1898-06-09,Italian,http://en.wikipedia.org/wiki/Luigi_Fagioli,Alfa Romeo,Swiss,http://en.wikipedia.org/wiki/Alfa_Romeo_in_For...,0
2,1950,1,reg_parnell,Reg Parnell,alfa,Alfa Romeo,4.0,3,3,4.0,4,70,Finished,+52.0,NaN,NaN,NaN,NaN,British Grand Prix,silverstone,Silverstone Circuit,1950-05-13,NaN,NaN,NaN,NaN,NaN,NaN,http://en.wikipedia.org/wiki/1950_British_Gran...,Silverstone Circuit,52.0786,-1.01694,Silverstone,UK,http://en.wikipedia.org/wiki/Silverstone_Circuit,Reg,Parnell,NaN,NaN,1911-07-02,British,http://en.wikipedia.org/wiki/Reg_Parnell,Alfa Romeo,Swiss,http://en.wikipedia.org/wiki/Alfa_Romeo_in_For...,0
3,1950,1,cabantous,Yves Cabantous,lago,Talbot-Lago,14.0,4,4,3.0,6,68,+2 Laps,NaN,NaN,NaN,NaN,NaN,British Grand Prix,silverstone,Silverstone Circuit,1950-05-13,NaN,NaN,NaN,NaN,NaN,NaN,http://en.wikipedia.org/wiki/1950_British_Gran...,Silverstone Circuit,52.0786,-1.01694,Silverstone,UK,http://en.wikipedia.org/wiki/Silverstone_Circuit,Yves,Cabantous,NaN,NaN,1904-10-08,French,http://en.wikipedia.org/wiki/Yves_Giraud_Caban...,Talbot-Lago,French,http://en.wikipedia.org/wiki/Talbot-Lago,0
4,1950,1,rosier,Louis Rosier,lago,Talbot-Lago,15.0,5,5,2.0,9,68,+2 Laps,NaN,NaN,NaN,NaN,NaN,British Grand Prix,silverstone,Silverstone Circuit,1950-05-13,NaN,NaN,NaN,NaN,NaN,NaN,http://en.wikipedia.org/wiki/1950_British_Gran...,Silverstone Circuit,52.0786,-1.01694,Silverstone,UK,http://en.wikipedia.org/wiki/Silverstone_Circuit,Louis,Rosier,NaN,NaN,1905-11-05,French,http://en.wikipedia.org/wiki/Louis_Rosier,Talbot-Lago,French,http://en.wikipedia.org/wiki/Talbot-Lago,0


# Create a new column "teamStrength" as the strength of the constructors over the years.

In [59]:
results['teamStrength'] = (
    results.groupby('constructorId')['position']
    .transform(lambda x: x.shift().expanding().mean())
)

In [60]:
results['teamStrength'].fillna(results['teamStrength'].mean(), inplace=True)

In [61]:
results.head()

,season,round,driverId,driverName,constructorId,constructorName_x,number,position,positionText,points,grid,laps,status,time_x,fastestLapRank,fastestLap_lap,fastestLapTime,averageSpeed,raceName,circuitId,circuitName_x,date,time_y,firstPractice,secondPractice,thirdPractice,qualifying,sprint,race_url,circuitName_y,lat,long,locality,country,url_y,givenName,familyName,code,permanentNumber,dateOfBirth,nationality_x,url_x,constructorName_y,nationality_y,url_y,driverExperience,teamStrength
0,1950,1,farina,Nino Farina,alfa,Alfa Romeo,2.0,1,1,9.0,1,70,Finished,2:13:23.6,NaN,NaN,NaN,NaN,British Grand Prix,silverstone,Silverstone Circuit,1950-05-13,NaN,NaN,NaN,NaN,NaN,NaN,http://en.wikipedia.org/wiki/1950_British_Gran...,Silverstone Circuit,52.0786,-1.01694,Silverstone,UK,http://en.wikipedia.org/wiki/Silverstone_Circuit,Nino,Farina,NaN,NaN,1906-10-30,Italian,http://en.wikipedia.org/wiki/Nino_Farina,Alfa Romeo,Swiss,http://en.wikipedia.org/wiki/Alfa_Romeo_in_For...,0,12.696735
1,1950,1,fagioli,Luigi Fagioli,alfa,Alfa Romeo,3.0,2,2,6.0,2,70,Finished,+2.6,NaN,NaN,NaN,NaN,British Grand Prix,silverstone,Silverstone Circuit,1950-05-13,NaN,NaN,NaN,NaN,NaN,NaN,http://en.wikipedia.org/wiki/1950_British_Gran...,Silverstone Circuit,52.0786,-1.01694,Silverstone,UK,http://en.wikipedia.org/wiki/Silverstone_Circuit,Luigi,Fagioli,NaN,NaN,1898-06-09,Italian,http://en.wikipedia.org/wiki/Luigi_Fagioli,Alfa Romeo,Swiss,http://en.wikipedia.org/wiki/Alfa_Romeo_in_For...,0,1.000000
2,1950,1,reg_parnell,Reg Parnell,alfa,Alfa Romeo,4.0,3,3,4.0,4,70,Finished,+52.0,NaN,NaN,NaN,NaN,British Grand Prix,silverstone,Silverstone Circuit,1950-05-13,NaN,NaN,NaN,NaN,NaN,NaN,http://en.wikipedia.org/wiki/1950_British_Gran...,Silverstone Circuit,52.0786,-1.01694,Silverstone,UK,http://en.wikipedia.org/wiki/Silverstone_Circuit,Reg,Parnell,NaN,NaN,1911-07-02,British,http://en.wikipedia.org/wiki/Reg_Parnell,Alfa Romeo,Swiss,http://en.wikipedia.org/wiki/Alfa_Romeo_in_For...,0,1.500000
3,1950,1,cabantous,Yves Cabantous,lago,Talbot-Lago,14.0,4,4,3.0,6,68,+2 Laps,NaN,NaN,NaN,NaN,NaN,British Grand Prix,silverstone,Silverstone Circuit,1950-05-13,NaN,NaN,NaN,NaN,NaN,NaN,http://en.wikipedia.org/wiki/1950_British_Gran...,Silverstone Circuit,52.0786,-1.01694,Silverstone,UK,http://en.wikipedia.org/wiki/Silverstone_Circuit,Yves,Cabantous,NaN,NaN,1904-10-08,French,http://en.wikipedia.org/wiki/Yves_Giraud_Caban...,Talbot-Lago,French,http://en.wikipedia.org/wiki/Talbot-Lago,0,12.696735
4,1950,1,rosier,Louis Rosier,lago,Talbot-Lago,15.0,5,5,2.0,9,68,+2 Laps,NaN,NaN,NaN,NaN,NaN,British Grand Prix,silverstone,Silverstone Circuit,1950-05-13,NaN,NaN,NaN,NaN,NaN,NaN,http://en.wikipedia.org/wiki/1950_British_Gran...,Silverstone Circuit,52.0786,-1.01694,Silverstone,UK,http://en.wikipedia.org/wiki/Silverstone_Circuit,Louis,Rosier,NaN,NaN,1905-11-05,French,http://en.wikipedia.org/wiki/Louis_Rosier,Talbot-Lago,French,http://en.wikipedia.org/wiki/Talbot-Lago,0,4.000000


# Load only the required rows for model training

In [62]:
results= results[results["season"] >= 2009]

In [63]:
results = results[results['position'] <= 20]

In [64]:
results = results[results['grid'] <= 20]

In [65]:
results.head(10)

,season,round,driverId,driverName,constructorId,constructorName_x,number,position,positionText,points,grid,laps,status,time_x,fastestLapRank,fastestLap_lap,fastestLapTime,averageSpeed,raceName,circuitId,circuitName_x,date,time_y,firstPractice,secondPractice,thirdPractice,qualifying,sprint,race_url,circuitName_y,lat,long,locality,country,url_y,givenName,familyName,code,permanentNumber,dateOfBirth,nationality_x,url_x,constructorName_y,nationality_y,url_y,driverExperience,teamStrength
19983,2009,1,button,Jenson Button,brawn,Brawn,22.0,1,1,10.0,1,58,Finished,1:34:15.784,3.0,17.0,1:28.020,216.891,Australian Grand Prix,albert_park,Albert Park Grand Prix Circuit,2009-03-29,06:00:00Z,NaN,NaN,NaN,NaN,NaN,http://en.wikipedia.org/wiki/2009_Australian_G...,Albert Park Grand Prix Circuit,-37.8497,144.968,Melbourne,Australia,http://en.wikipedia.org/wiki/Melbourne_Grand_P...,Jenson,Button,BUT,22.0,1980-01-19,British,http://en.wikipedia.org/wiki/Jenson_Button,Brawn,British,http://en.wikipedia.org/wiki/Brawn_GP,9,12.696735
19984,2009,1,barrichello,Rubens Barrichello,brawn,Brawn,23.0,2,2,8.0,2,58,Finished,+0.807,14.0,43.0,1:29.066,214.344,Australian Grand Prix,albert_park,Albert Park Grand Prix Circuit,2009-03-29,06:00:00Z,NaN,NaN,NaN,NaN,NaN,http://en.wikipedia.org/wiki/2009_Australian_G...,Albert Park Grand Prix Circuit,-37.8497,144.968,Melbourne,Australia,http://en.wikipedia.org/wiki/Melbourne_Grand_P...,Rubens,Barrichello,BAR,NaN,1972-05-23,Brazilian,http://en.wikipedia.org/wiki/Rubens_Barrichello,Brawn,British,http://en.wikipedia.org/wiki/Brawn_GP,16,1.000000
19985,2009,1,trulli,Jarno Trulli,toyota,Toyota,9.0,3,3,6.0,20,58,Finished,+1.604,10.0,50.0,1:28.916,214.706,Australian Grand Prix,albert_park,Albert Park Grand Prix Circuit,2009-03-29,06:00:00Z,NaN,NaN,NaN,NaN,NaN,http://en.wikipedia.org/wiki/2009_Australian_G...,Albert Park Grand Prix Circuit,-37.8497,144.968,Melbourne,Australia,http://en.wikipedia.org/wiki/Melbourne_Grand_P...,Jarno,Trulli,TRU,NaN,1974-07-13,Italian,http://en.wikipedia.org/wiki/Jarno_Trulli,Toyota,Japanese,http://en.wikipedia.org/wiki/Toyota_Racing,12,11.711382
19986,2009,1,glock,Timo Glock,toyota,Toyota,10.0,4,4,5.0,19,58,Finished,+4.435,6.0,53.0,1:28.416,215.920,Australian Grand Prix,albert_park,Albert Park Grand Prix Circuit,2009-03-29,06:00:00Z,NaN,NaN,NaN,NaN,NaN,http://en.wikipedia.org/wiki/2009_Australian_G...,Albert Park Grand Prix Circuit,-37.8497,144.968,Melbourne,Australia,http://en.wikipedia.org/wiki/Melbourne_Grand_P...,Timo,Glock,GLO,NaN,1982-03-18,German,http://en.wikipedia.org/wiki/Timo_Glock,Toyota,Japanese,http://en.wikipedia.org/wiki/Toyota_Racing,5,11.676113
19987,2009,1,alonso,Fernando Alonso,renault,Renault,7.0,5,5,4.0,10,58,Finished,+4.879,9.0,53.0,1:28.712,215.199,Australian Grand Prix,albert_park,Albert Park Grand Prix Circuit,2009-03-29,06:00:00Z,NaN,NaN,NaN,NaN,NaN,http://en.wikipedia.org/wiki/2009_Australian_G...,Albert Park Grand Prix Circuit,-37.8497,144.968,Melbourne,Australia,http://en.wikipedia.org/wiki/Melbourne_Grand_P...,Fernando,Alonso,ALO,14.0,1981-07-29,Spanish,http://en.wikipedia.org/wiki/Fernando_Alonso,Renault,French,http://en.wikipedia.org/wiki/Renault_in_Formul...,8,10.515723
19988,2009,1,rosberg,Nico Rosberg,williams,Williams,16.0,6,6,3.0,5,58,Finished,+5.722,1.0,48.0,1:27.706,217.668,Australian Grand Prix,albert_park,Albert Park Grand Prix Circuit,2009-03-29,06:00:00Z,NaN,NaN,NaN,NaN,NaN,http://en.wikipedia.org/wiki/2009_Australian_G...,Albert Park Grand Prix Circuit,-37.8497,144.968,Melbourne,Australia,http://en.wikipedia.org/wiki/Melbourne_Grand_P...,Nico,Rosberg,ROS,6.0,1985-06-27,German,http://en.wikipedia.org/wiki/Nico_Rosberg,Williams,British,http://en.wikipedia.org/wiki/Williams_Grand_Pr...,3,9.727009
19989,2009,1,buemi,Sébastien Buemi,toro_rosso,Toro Rosso,12.0,7,7,2.0,13,58,Finished,+6.004,16.0,34.0,1:29.230,213.950,Australian Grand Prix,albert_park,Albert Park Grand Prix Circuit,2009-03-29,06:00:00Z,NaN,NaN,NaN,NaN,NaN,http://en.wikipedia.org/wiki/2009_Australi

In [66]:
results['position'].max()

np.int64(20)

In [67]:
results['grid'].max()

np.int64(20)

## Create new csv with merged dataset for data preprocessing

In [68]:
results.to_csv('unclean_results.csv', index=False)